# Exercise-2
Write an implementation of a rainbow table attack against the  
Microsoft LAN Manager password-hashing scheme.

In [1]:
from lmdes import lmdes, bitarray2str
from rainbow_lm import RainbowTable
from bitarray import bitarray

In [2]:
bit_array = bitarray()
for i in range(7 * 8):
    bit_array.append(0)
print(bit_array)
pwd_zero = bitarray2str(bit_array)

bitarray('00000000000000000000000000000000000000000000000000000000')


In [3]:
word = "17437184211106"
target = lmdes(word).generate_hash()
h1, h2 = target[:16], target[16:]
print("Target hash:", target)
print("Hash h1:", h1)
print("Hash h2:", h2)

Target hash: 003d4aaaf03ecf32ed2deb668b4208b6
Hash h1: 003d4aaaf03ecf32
Hash h2: ed2deb668b4208b6


In [4]:
numbers = '0123456789'
rain = RainbowTable(columns=10000, chars=numbers,rows=3048)

In [5]:
rain.__repr__
rain.writeToFile("rain_num_R=3048_C=10000.txt")

In [6]:
# rain = RainbowTable()
# rain.readFromFile("rain_num_R=20000_C=200.txt")
# rain.__repr__

In [7]:
pwd1 = rain.crackHash(h1)

In [9]:
# if h2 == 'aad3b435b51404ee':
#     pwd2 = pwd_zero
# else:  
#     print("Password longer than 7 characters")
#     pwd2 = rain.crackHash(h2)

In [10]:
print("Cracked password (1): %r" % (pwd1))
# print("Cracked password (2): %r" % (pwd2))

Cracked password (1): '1743718'


In [ ]:
# rain.writeToFile("rain_num.txt")